# Analysis
* Create yml for all my files. 
* Save overlay? 

In [ ]:
 21QA21QWADzcazcAzcXVazcf# Read in zip files
# Graphs
import altair as alt
import fsspec
import geopandas as gpd
import intake
import numpy as np
import pandas as pd

# My utilities
import utilities
from calitp import *

# Display
from IPython.display import HTML, Image, Markdown, display, display_html

# Geometry
from shared_utils import geography_utils, utils

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/cellular_coverage/"

In [ ]:
routes_df = utilities.load_unique_routes_df()

In [ ]:
routes_df.shape

In [ ]:
routes_df.columns

In [ ]:
suffix = "test"

In [ ]:
routes_df2 = routes_df.rename(
    columns={
        c: c + suffix
        for c in routes_df.columns
        if c in ["geometry", "route_type", "route_name"]
    }
)

In [ ]:
routes_df2.columns

## T-Mobile

In [ ]:
tmobile_df = gpd.read_parquet(f"{GCS_FILE_PATH}tmobile_california.parquet")

In [ ]:
tmobile_m1 = utilities.route_cell_coverage(tmobile_df, routes_df, "_tmobile")

In [ ]:
type(tmobile_m1)

In [ ]:
tmobile_m1.shape

In [ ]:
tmobile_m1.columns

In [ ]:
(tmobile_m1["binned_tmobile"].value_counts() / len(tmobile_m1)) * 100

## AT&T

In [ ]:
att_df = utilities.load_att()

In [ ]:
att_m1 = utilities.route_cell_coverage(att_df, routes_df, "_att")

In [ ]:
type(att_m1)

In [ ]:
att_m1.shape

In [ ]:
(att_m1["binned_att"].value_counts() / len(att_m1)) * 100

## Verizon

In [ ]:
verizon_df = utilities.load_verizon()

In [ ]:
verizon_m1 = utilities.route_cell_coverage(verizon_df, routes_df, "_verizon")

In [ ]:
type(verizon_m1)

In [ ]:
verizon_m1.shape

In [ ]:
(verizon_m1["binned_verizon"].value_counts() / len(verizon_m1)) * 100

In [ ]:
verizon_m1.columns

## Merge all three providers

In [ ]:
common_cols = ["route_id", "route_name", "agency", "itp_id", "route_length_original_df"]

In [ ]:
merge1 = att_m1.merge(verizon_m1, how="inner", on=common_cols).merge(
    tmobile_m1, how="outer", on=common_cols
)

In [ ]:
# Del extra cols that are the same across dfs.
merge1 = merge1.drop(
    columns=[
        "binned_verizon",
        "binned_att",
        "binned_tmobile",
    ]
)

In [ ]:
# Ensure this is a geometry col
merge1 = gpd.GeoDataFrame(merge1, geometry= 'geometry_overlay_verizon', crs="EPSG:4326")

In [ ]:
type(merge1)

In [ ]:
merge1.shape

### Some checks
* Original df against merged.

In [ ]:
# 1 extra row missing??
routes_df.shape, merge1.shape

In [ ]:
# Find the missing row
route_names_og = set(routes_df.route_name.unique().tolist())
route_names_merge = set(merge1.route_name.unique().tolist())
route_names_og - route_names_merge

In [ ]:
routes_df.route_id.nunique(), merge1.route_id.nunique()

In [ ]:
routes_df.itp_id.nunique(), merge1.itp_id.nunique()

### Filter out for low threshold 
* Chose 70% or less cell coverage as the threshold.
* Most routes across the 3 providers have 90% or more coverage.
* Verizon has the best coverage. 85% of the routes have 90% or more coverage with Verizon. 

In [ ]:
# Merge low att & verizon coverage
m1 = low_att_coverage.merge(
    low_verizon_coverage,
    how="outer",
    left_on=["route_id_att", "route_name_att", "agency_att", "itp_id_att"],
    right_on=[
        "route_id_verizon",
        "route_name_verizon",
        "agency_verizon",
        "itp_id_verizon",
    ],
    indicator=True,
)

In [ ]:
m1["_merge"].value_counts()

In [ ]:
# Turn to gpd
m1 = gpd.GeoDataFrame(m1, geometry="geometry_overlay_att", crs="EPSG:4326")

In [ ]:
type(m1)

In [ ]:
""" m1.loc[m1["_merge"] == "both"][
    ["route_name_att", "route_name_verizon", "percentage_verizon", "percentage_att"]
].drop_duplicates(subset=["route_name_att"]) """

#### Check a left only value for Verizon
* Left only routes have more than 70% coverage through Verizon.

In [ ]:
verizon_m1.loc[verizon_m1["route_name_verizon"] == "Ridgecrest Shuttle"][
    [
        "route_name_verizon",
        "percentage_verizon",
    ]
]

In [ ]:
verizon_m1.loc[verizon_m1["route_name_verizon"] == "Mainline AM/PM"][
    [
        "route_name_verizon",
        "percentage_verizon",
    ]
]

In [ ]:
# Filter out for only both, these are routes that really dont have any more than 60% cellular coverage among AT&T and Verizon
m1 = m1.loc[m1["_merge"] == "both"]

In [ ]:
# Drop unwanted cols
m1 = m1.drop(
    columns=[
        "_merge",
        "route_type_att",
        "binned_att",
        "route_id_verizon",
        "route_name_verizon",
        "agency_verizon",
        "itp_id_verizon",
        "geometry_overlay_verizon",
        "route_length_overlay_verizon",
        "geometry_original_df_verizon",
        "route_type_verizon",
        "route_length_original_df_verizon",
        "binned_verizon",
    ]
)

In [ ]:
m1.shape

In [ ]:
# Merge att & verizon coverage
m2 = pd.merge(
    m1,
    low_tmobile_coverage,
    how="outer",
    left_on=["route_id_att", "route_name_att", "agency_att", "itp_id_att"],
    right_on=[
        "route_id_tmobile",
        "route_name_tmobile",
        "agency_tmobile",
        "itp_id_tmobile",
    ],
    indicator=True,
)

In [ ]:
# Make sure it remains a GDF
m1 = gpd.GeoDataFrame(m1, geometry="geometry_overlay_att", crs="EPSG:4326")

In [ ]:
m2["_merge"].value_counts()

In [ ]:
# Filter out for only both, these are routes that really dont have any more than treshold % cellular coverage among all 3
m3 = m2.loc[m2["_merge"] == "both"]

In [ ]:
m3 = m3.drop(
    columns=[
        "route_id_tmobile",
        "route_name_tmobile",
        "agency_tmobile",
        "itp_id_tmobile",
        "geometry_overlay_tmobile",
        "route_length_overlay_tmobile",
        "geometry_original_df_tmobile",
        "route_type_tmobile",
        "route_length_original_df_tmobile",
        "binned_tmobile",
        "_merge",
    ]
)

In [ ]:
# preview df
m3.drop(columns=["geometry_original_df_att", "geometry_overlay_att"]).sample(3)

In [ ]:
threshold = 70

In [ ]:
# Only keep routes that have less than threshold across all 3 providers.
low_coverage = merge1[
    (merge1["percentage_verizon"] < threshold)
    & (merge1["percentage_tmobile"] < threshold)
    & (merge1["percentage_att"] < threshold)
]

In [ ]:
# Create a column with median of all three providers
low_coverage["median_cell_coverage_percentage"] = low_coverage[
    ["percentage_verizon", "percentage_tmobile", "percentage_att"]
].mean(axis=1)

In [ ]:
len(low_coverage), type(low_coverage), low_coverage.route_id.nunique()

In [ ]:
low_coverage.columns

In [ ]:
# (low_coverage.loc[:,~low_coverage.columns.str.contains('^geometry', case=False)]).head(3)

## Add trips
* Add one column for total trips that an agency completes across all routes.
* Add another column for total trips ONLY for routes with low coverage. 

In [ ]:
trips  = utilities.trip_df()

In [ ]:
trips.columns

In [ ]:
# Merge the 2 filtered at&t and verizon dataframes so we can see which routes overlap
trips_routes = pd.merge(
    low_coverage,
    trips,
    how="left",
    left_on=["itp_id", "route_id"],
    right_on=["calitp_itp_id", 'route_id'],
    indicator=True,
)

In [ ]:
trips_routes["_merge"].value_counts()

In [ ]:
# Add column for percentage of trips for the route in question compared with
# All the trips an agency has done
trips_routes["percentage_of_trips_w_low_cell_service"] = (
    trips_routes["total_trips_by_route"] / trips_routes["total_trips_by_agency"]
)

In [ ]:
trips_routes = trips_routes.drop(columns=["_merge", "calitp_itp_id"])

In [ ]:
trips_routes.shape

## Add NTD

In [ ]:
# Load NTD vehicles
ntd_df = utilities.ntd_vehicles()[["agency", "total_buses"]]

In [ ]:
# Replace agency names in NTD to match routes_df above
ntd_df["agency"] = ntd_df["agency"].replace(
    {
        "Trinity County": "Trinity Transit",
        "City of Calabasas": "Calabasas Transit System",
        "County of Sonoma": "Sonoma County Transit",
        "Tehama County": "Tehama Rural Area eXpress",
        "Los Angeles County Department of Public Works - East L.A.": "East Los Angeles Shuttle",
        "Sacramento Regional Transit District": "Sacramento Regional Transit District",
        "Eastern Sierra Transit Authority": "Mammoth Lakes Transit System",
        "City of Lompoc": "City of Lompoc Transit",
        "San Luis Obispo Regional Transit Authority": "South County Transit Link",
        "City of Roseville": "Roseville Transit",
        "Los Angeles County Dept. of Public Works - Athens Shuttle Service": "the Link-Athens",
        "Los Angeles County Department of Public Works - Avocado Heights": "Avocado Heights/Bassett/West Valinda Shuttle",
        "Susanville Indian Rancheria": "Susanville Indian Rancheria Public Transportation Program",
        "Transit Joint Powers Authority for Merced County": "Merced The Bus",
        "City of Eureka": "Eureka Transit Service",
        "Nevada County Transit Services": "Gold Country Stage",
        "San Mateo County Transit District": "SamTrans",
        "Redwood Coast Transit Authority": "Redwood Coast Transit",
        "City of Avalon": "Avalon Transit",
        "City of Lodi": "Grapeline",
        "Golden Gate Bridge": "Golden Gate Bridge Highway and Transportation District",
        "City of Santa Maria": "Santa Maria Area Transit",
    }
)

In [ ]:
# Replace agency names in agency4 to match ntd
trips_routes["agency"] = trips_routes["agency"].replace(
    {
        "Cloverdale Transit": "Sonoma County Transit",
    }
)

In [ ]:
trips_routes_ntd = pd.merge(
    trips_routes,
    ntd_df,
    how="left",
    on="agency",
    indicator=True,
)

In [ ]:
# Create col to find number of buses that run in low cell coverage routes.
trips_routes_ntd["estimate_of_buses_in_low_cell_zones"] = (
    (trips_routes_ntd["total_buses"] * trips_routes_ntd["percentage_of_trips_w_low_cell_service"])
    .fillna(0)
    .astype("int64")
)

## Final

### There are a lot of same route-id and same route-name combos but run by different agencies.
* Scanned [agencies](https://github.com/cal-itp/data-infra/blob/main/airflow/data/agencies.yml) to make sure all duplicates lead to the same gtsf schedule url.
* If so, del duplicates.

In [ ]:
trips_routes_ntd.route_id.value_counts()

In [ ]:
# Del duplicates based on whichever agency with the same route has the most buses.
final = trips_routes_ntd.sort_values(
    ["route_id", "route_name", "total_buses"], ascending=[True, True, False]
).drop_duplicates(subset=["route_id", "route_name"])

In [ ]:
final.shape, type(final)

In [ ]:
final= final.set_geometry('geometry_overlay_att')

### Some of the results are surprising...wouldn't expect urban areas to have routes on this list

In [ ]:
# https://stackoverflow.com/questions/38383886/drop-column-based-on-a-string-condition
final.loc[:, ~final.columns.str.contains('route_length|geometry|_merge')]

In [ ]:
final.explore(
    "route_name",
    width=800,
    height=400,
    tooltip=["route_name"],
    style_kwds={"weight": 6},
    legend=False,
    color="tab20c",
)

In [ ]:
# Grab route ID&agency
final_routes = final[["route_id", "agency"]]

In [ ]:
routes_df2 = pd.merge(routes_df, final_routes, how="inner", on=["route_id", "agency"])

In [ ]:
routes_df2.columns

In [ ]:
routes_df2.explore(
    "route_length",
    width=800,
    height=400,
    tooltip=["route_name", "agency"],
    style_kwds={"weight": 6},
    legend=False,
    color="tab20c",
)